In [ ]:
# Given a credit card csv file in Nationwide (UK) format,
# generate a csv format acceptable to Homebank.
# Nationwide excludes location data from its ofx output,
# hence no ofx conversion here.
# Given the total absence of good practice here, there are 
# many todos.

# include these in requirements.txt:
import pandas as pd
import pprint

# Note the encoding as delivered by Nationwide
nw_encoding = 'cp1252'
input_df = pd.read_csv("c:/temp/aa4.csv", skiprows=4, encoding=nw_encoding)


# Process data and create the output DataFrame
output_df = pd.DataFrame({
    "Date": input_df["Date"],
    "payment type": 1,  # placeholder
    "info": "info 1",  # placeholder
    "Transactions": input_df["Transactions"],
    "Location": input_df["Location"],
    "Paid out": input_df["Paid out"].astype(str).str.replace("£", ""),
    "Paid in": input_df["Paid in"].astype(str).str.replace("£", ""),
    "category": "category 1",  # placeholder
    "Tags": "tag1 tag2"  # placeholder
})

# Do the more complex stuff in a second pass
output_df.fillna(0,inplace=True)
output_df["Paid out"] = output_df["Paid out"].astype(float)
output_df["Paid in"] = output_df["Paid in"].astype(float)
output_df["Paid"] = output_df.apply(lambda row: (row["Paid out"] * -1) if row["Paid out"] > 0.0 else row["Paid in"], axis = 1)
output_df["Date"] = pd.to_datetime(output_df["Date"]).dt.strftime("%y-%m-%d")

# Finally in a form that HomeBank can ingest:
columns_to_delete = ["Paid out", "Paid in"] 
output_df.drop(columns=columns_to_delete, inplace=True)
desired_order = [0,1,2,3,4,7,5,6]
output_df = output_df.iloc[:, desired_order]

pprint.pprint(output_df) # debug

output_df.to_csv("output.csv", sep=";", index=False, header=False)

print("got to the end")
